<a href="https://colab.research.google.com/github/fatemafaria142/Comparative-Analysis-of-Diverse-Large-Language-Models-in-Story-Generation/blob/main/Story_Generation_using_Mistral_7B_Instruct_v0_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install Required Packages**

In [1]:
!pip install accelerate peft bitsandbytes transformers trl datasets torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00


#### **Dataset Link:** https://huggingface.co/datasets/AtlasUnified/atlas-storyteller

In [2]:
from datasets import load_dataset

instruct_tune_dataset = load_dataset("AtlasUnified/atlas-storyteller")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

### **Dataset structure**
* The dataset contains two columns.

In [3]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'Story'],
        num_rows: 5018
    })
})

In [4]:
# Display information for 5 data points from the 'train' split
num_samples_to_show = 5
for i in range(num_samples_to_show):
    data = instruct_tune_dataset['train'][i]
    print(f"Data Point {i + 1}:")
    print("ID:", data['id'])
    print("Story:", data['Story'])
    print("\n-----------------------------\n")

Data Point 1:
ID: seed_task_0
Story: In the bustling city of New York, where the neon lights flickered and the sound of traffic reverberated through the streets, lived a man named Ethan Sullivan. With his chiseled jawline, piercing blue eyes, and a physique carved by years of training, Ethan possessed an air of mystery that intrigued those who crossed his path. He lived a solitary life in a modest apartment, spending his days as an accountant, his nights cloaked in shadows and secrecy.  One fateful morning, as the city awoke to the rhythmic beat of its own heartbeat, Ethan received a cryptic message on a burner phone. It simply stated, "They're coming for you." His heart quickened, and a wave of apprehension washed over him. Who were "they," and why were they after him? Ethan's blood ran cold, fueling a surge of adrenaline that urged him to take action.  Without a moment's hesitation, Ethan gathered his meager belongings and made his way to a hidden room beneath his apartment. The room

### **We will use just a small subset of the data for this training example**

In [5]:
instruct_tune_dataset["train"] = instruct_tune_dataset["train"].select(range(3500))
instruct_tune_dataset["test"] = instruct_tune_dataset["train"].select(range(300))

In [6]:
instruct_tune_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'Story'],
        num_rows: 3500
    })
    test: Dataset({
        features: ['id', 'Story'],
        num_rows: 300
    })
})

* Note that this time, the tokenizer has added the control tokens `[INST]` and `[/INST]` to indicate the start and end of user messages (but not assistant messages!). **Mistral-instruct was trained with these tokens.**
* In order to leverage instruction fine-tuning, your prompt should be surrounded by `[INST]` and `[/INST]` tokens. The very first instruction should begin with a begin of sentence id. The next instructions should not. The assistant generation will be ended by the end-of-sentence token id.

In [10]:
def create_prompt(sample):
    """
    Update the prompt template:
    Combine both the prompt and input into a single column.
    """
    bos_token = "<s>"
    eos_token = "</s>"

    # Use a predefined template for instructions
    instructions_template = "Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story! "
    full_prompt = ""
    full_prompt += bos_token
    full_prompt += "[INST]"
    full_prompt += instructions_template
    full_prompt += sample['Story']
    full_prompt += "[/INST]"
    full_prompt += eos_token

    return full_prompt

In [11]:
create_prompt(instruct_tune_dataset["train"][0])

'<s>[INST]Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story! In the bustling city of New York, where the neon lights flickered and the sound of traffic reverberated through the streets, lived a man named Ethan Sullivan. With his chiseled jawline, piercing blue eyes, and a physique carved by years of training, Ethan possessed an air of mystery that intrigued those who crossed his path. He lived a solitary life in a modest apartment, spending his days as an accountant, his nights cloaked in shadows and secrecy.  One fateful morning, as the city awoke to the rhythmic beat of its own heartbeat, Ethan received a cryptic message on a burner phone. It simply stated, "They\'re coming for you." His heart quickened, and a wave of apprehension washed over him. Who were "they," and why were they a

In [12]:
create_prompt(instruct_tune_dataset["train"][1])

'<s>[INST]Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story! The sky stretched out before them, an endless expanse of blue pierced only by fluffy white clouds. The air crackled with the anticipation of action as the squadron gathered at the airfield, their sleek fighter jets gleaming under the noon sun. Among the pilots was Captain Joshua Bennett, a seasoned veteran with eyes as sharp as a hawk\'s, and a heart that beat with the rhythm of war.  Set in the midst of a raging conflict, the story takes place during the height of World War II. Captain Bennett, a fearless and daring leader, had earned his reputation through countless successful missions. He was known for his audacious maneuvers and masterful control of aircraft, traits that instilled confidence in every member of his squadro

In [13]:
create_prompt(instruct_tune_dataset["train"][2])

'<s>[INST]Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story! In the midst of a bustling metropolis teeming with skyscrapers that seemingly touched the heavens, nestled a small and unassuming garage. It belonged to Ethan, a young and brilliant engineer consumed by his insatiable thirst for innovation and adventure. With grease-stained hands and a mind full of ideas, he spent every waking hour tinkering and crafting marvelous inventions that could turn dreams into reality.  One fateful afternoon, while rummaging through a box filled with ancient blueprints, Ethan stumbled upon a peculiar schematic. It depicted a colossal machine, a menacing giant robot with gleaming steel limbs and an aura of unstoppable power. Intrigued and fascinated, he couldn\'t resist the temptation to bring this me

In [14]:
create_prompt(instruct_tune_dataset["train"][3])

"<s>[INST]Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story! In the heart of a once-thriving metropolis, now reduced to a desolate wasteland, a lone figure emerged from the shadows. Leather-clad and armed to the teeth, his name was Gabriel. Standing at the precipice of an abandoned street, he surveyed the remnants of what was once a bustling city. Hollowed buildings loomed like guardians of a forgotten time, their broken windows reflecting the glimmers of an apocalyptic twilight.  It had been years since the outbreak, the virus that turned ordinary people into ravenous zombies. Gabriel, a former army officer, had become a symbol of hope in this desolate world. With his unwavering determination and unparalleled combat skills, he had earned the respect of fellow survivors. But there was 

In [15]:
create_prompt(instruct_tune_dataset["train"][4])

'<s>[INST]Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story! In the gritty underbelly of a city plagued by crime, Detective John Anderson was a tenacious force to be reckoned with. Tall and lean, with a sharp jawline and piercing blue eyes, he exuded an aura of unwavering determination. The city had become his battleground, and the criminals feared his name.  The whispers on the streets spoke of a notorious criminal mastermind known as "The Phantom." No one had ever seen his face, and his crimes left the police baffled. Like a phantom, he slipped through their fingers, leaving no trace behind. Detective Anderson was obsessed with catching him, a relentless pursuit that consumed his every waking moment.  Late one stormy evening, as the rain poured mercilessly, Detective Anderson receive

In [16]:
create_prompt(instruct_tune_dataset["train"][5])

"<s>[INST]Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story! In the crumbling metropolis of Neo-Arkadia, a city teetering on the brink of chaos, rival gangs waged war for control over scarce resources. Dilapidated skyscrapers stood tall, their shattered windows reflecting the desolation that shrouded the once-thriving urban landscape. Among this dystopian backdrop, two formidable gangs emerged as the most powerful contenders: the Razor Wings and the Shadow Serpents.  The city's unmistakable staleness hung in the air as the sun dipped below the horizon, casting long shadows across the cracked pavement. The Red Sector, a territory claimed by the Shadow Serpents, pulsated with an otherworldly energy. Led by the enigmatic and ruthless Viper, the gang thrived in the darkness, leaving a trai

### **Initializing the Model**
* Load the model using a 4-bit configuration, employing double quantization, and set bfloat16 as the compute data type.

* Notably, we opt for the instruct-tuned model in this instance rather than the base model. It's worth mentioning that fine-tuning a base model necessitates a more substantial amount of data!

In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
    )


* **Model and Tokenizer:** https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2

In [18]:
mode_id = "mistralai/Mistral-7B-Instruct-v0.2"

In [19]:
model = AutoModelForCausalLM.from_pretrained(
        mode_id, quantization_config=bnb_config, device_map="auto", use_cache=False
    )

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [20]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

### **Let's example how well the model does at this task currently:**

In [22]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1024, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0].replace(prompt, "")

In [23]:
# Use a predefined template for instructions
prompt = "<s>[INST] Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story! "
prompt += "Amidst the inky blackness of the night, a seamless curtain of shadows unfurled itself over the dense wilderness. In the heart of a war-torn nation, nestled between towering mountains and treacherous valleys, lay a remote enemy stronghold. [/INST]"
print(prompt)

<s>[INST] Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story! Amidst the inky blackness of the night, a seamless curtain of shadows unfurled itself over the dense wilderness. In the heart of a war-torn nation, nestled between towering mountains and treacherous valleys, lay a remote enemy stronghold. [/INST]


In [24]:
generate_response(prompt, model)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


'<s><s> [INST] Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story! Amidst the inky blackness of the night, a seamless curtain of shadows unfurled itself over the dense wilderness. In the heart of a war-torn nation, nestled between towering mountains and treacherous valleys, lay a remote enemy stronghold. [/INST] As the moon waned, a stealthy figure emerged from the depths of the forest, clutching a tattered map in his calloused hands. His name was Renn, a seasoned scout and survivor of the ongoing conflict. His eyes scanned the terrain, searching for any signs of guards or traps that might bar his way to the stronghold. He had spent weeks gathering intelligence and finally, the moment had arrived.\n\nRenn was a member of the clandestine resistance group, the Shattered Shield, dedicated 

### **Setting up the Training**
we will be using the `huggingface` and the `peft` library!

In [25]:
from peft import AutoPeftModelForCausalLM, LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, bias="none", task_type="CAUSAL_LM")


* we need to prepare the model to be trained in 4bit so we will use the  **`prepare_model_for_kbit_training`** function from peft




In [26]:
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# **Training Hyperparameters**
The choice of hyperparameters is contingent upon the desired training duration. Pay special attention to the following key factors:

* `num_train_epochs/max_steps:` Dictates the number of iterations over the data. Exercise caution, as an excessive number may lead to overfitting!

* `learning_rate:` Governs the convergence speed of the model. Adjust this parameter judiciously for optimal results.

In [28]:
from transformers import TrainingArguments
output_model= "mistral_instruct_generation"
training_arguments = TrainingArguments(
        output_dir=output_model,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=10,
        num_train_epochs=1,
        max_steps=100,
        fp16=True,
)


### **Setting up the trainer**

`max_seq_length`: Context window size


In [29]:
from trl import SFTTrainer

max_seq_length = 1024

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt, # this will aplly the create_prompt mapping to all training and test dataset
  args=training_arguments,
  train_dataset=instruct_tune_dataset["train"],
  eval_dataset=instruct_tune_dataset["test"]
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:304: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


### **Training starts here**

In [30]:
trainer.train()

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.630900
20,1.369500
30,1.320400
40,1.312500
50,1.270000
60,1.264300
70,1.252000
80,1.244500
90,1.246700
100,1.272000


TrainOutput(global_step=100, training_loss=1.3182857894897462, metrics={'train_runtime': 1664.8881, 'train_samples_per_second': 0.24, 'train_steps_per_second': 0.06, 'total_flos': 1.74835334381568e+16, 'train_loss': 1.3182857894897462, 'epoch': 0.11})

### **Save the model**

In [31]:
trainer.save_model("mistral_instruct_generation")

In [32]:
merged_model = model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:229: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [33]:
def generate_response(prompt, model):
  encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
  model_inputs = encoded_input.to('cuda')

  generated_ids = model.generate(**model_inputs, max_new_tokens=1024, do_sample=True, pad_token_id=tokenizer.eos_token_id)

  decoded_output = tokenizer.batch_decode(generated_ids)

  return decoded_output[0]

### **Example No:1**

In [34]:
# Use a predefined template for instructions
prompt = '''<s>[INST] Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story! '''
prompt += '''In the crumbling metropolis of Neo-Arkadia, a city teetering on the brink of chaos, rival gangs waged war for control over scarce resources. Dilapidated skyscrapers stood tall, their shattered windows reflecting the desolation that shrouded the once-thriving urban landscape. [/INST]'''
response = generate_response(prompt, model)
# Print the response with formatted output
print(response)

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s><s> [INST] Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story! In the crumbling metropolis of Neo-Arkadia, a city teetering on the brink of chaos, rival gangs waged war for control over scarce resources. Dilapidated skyscrapers stood tall, their shattered windows reflecting the desolation that shrouded the once-thriving urban landscape. [/INST] Amidst the cacophony of gunfire and explosions, a lone figure emerged from the shadows. His name was Kane, a charismatic leader with piercing blue eyes and a scar that slashed across his left cheek. He commanded the respect of his followers, a motley crew of survivors, bound together by the shared belief that in Neo-Arkadia, only the strong could survive.

Kane's rivals were the Maliks, led by a ruthless and brutal warlord known only as The Vi

### **Example No:2**

In [35]:
# Use a predefined template for instructions
prompt = '''<s>[INST] Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story! '''
prompt += '''In the gritty underbelly of a city plagued by crime, Detective John Anderson was a tenacious force to be reckoned with. Tall and lean, with a sharp jawline and piercing blue eyes, he exuded an aura of unwavering determination. The city had become his battleground, and the criminals feared his name. [/INST]'''
response = generate_response(prompt, model)
# Print the response with formatted output
print(response)

<s><s> [INST] Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story! In the gritty underbelly of a city plagued by crime, Detective John Anderson was a tenacious force to be reckoned with. Tall and lean, with a sharp jawline and piercing blue eyes, he exuded an aura of unwavering determination. The city had become his battleground, and the criminals feared his name. [/INST] As the sun sets over the city, Detective John Anderson leaves the precinct, his mind heavy with the weight of unsolved cases. The streets are slick with rain, the neon signs reflecting off the wet cobblestones, casting long, dancing shadows. John's gut tells him that something is amiss. He hails a cab and asks the driver to take him to the docks.  Arriving at the docks, John is greeted by the familiar sight of rusted sh

### **Example No:3**

In [36]:
# Use a predefined template for instructions
prompt = '''<s>[INST] Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story! '''
prompt += '''The sky stretched out before them, an endless expanse of blue pierced only by fluffy white clouds. The air crackled with the anticipation of action as the squadron gathered at the airfield, their sleek fighter jets gleaming under the noon sun. [/INST]'''
response = generate_response(prompt, model)
# Print the response with formatted output
print(response)

<s><s> [INST] Imagine you are the author of this story. Your task is to continue the narrative and unfold the plot. Introduce new characters, unexpected twists, and exciting events. Feel free to unleash your creativity and have fun crafting the next part of the story! The sky stretched out before them, an endless expanse of blue pierced only by fluffy white clouds. The air crackled with the anticipation of action as the squadron gathered at the airfield, their sleek fighter jets gleaming under the noon sun. [/INST] Among the pilots, a young woman named Ava stood out. Her eyes were filled with determination and her posture was confident. She had earned her place in the squadron through sheer grit and hard work, and was determined to prove herself a formidable fighter. Her comrades greeted her with respect and camaraderie, their faces reflecting a shared sense of purpose.

As the squadron took off into the vast expanse of the sky, Ava's heart raced with excitement. She climbed higher and